In [22]:
import requests
from time import sleep
from pprint import pprint
import json
from tqdm import tqdm
from datetime import datetime
from configparser import ConfigParser

In [16]:
# Функция формирования json файла
def json_photos(file_name, lst):
    file_name = f'{file_name}.json'
    result = []
    for i in lst:
        result.append({
            'file_name': i['file_name'],
            'size': i['size']
        })
    json_string = json.dumps(result, indent=4)
    with open(file_name, 'w') as json_file:
        json_file.write(json_string)


# VK API
class VkGet:
    url = 'https://api.vk.com/method/'
    def __init__(self, vk_token, version = '5.131'):
         self.params = {
            'access_token': vk_token,
            'v': version
            }
    
    def get_user_id(self, str):
        """Метод получает id профиля по screen_name """
        get_user_id_params = {
            'screen_name': str
        }
        get_user_id_url = self.url + 'utils.resolveScreenName'
        response = requests.get(get_user_id_url, params={**self.params, **get_user_id_params}).json()
        return response['response']['object_id'] 

    def get_photos(self, owner_id = None, album_id = 'profile', count = 5):
        """Метод получает фотографии с профиля, по умолчанию - аватарки владельца профиля в количестве 5 штук.
            Возращает массив в формате json """
        get_photos_params = {
            'owner_id': owner_id,
            'album_id':  album_id,
            'rev': 1,
            'extended': 1,
            'count': count
            }
        get_photos_url = self.url + 'photos.get'
        response = requests.get(get_photos_url, params={**self.params, **get_photos_params}).json()
        return response['response']['items']

    def lst_photos(self, lst):
        """Метод формирует список с необходимой для дальнейшей работы информацией о фотографиях"""
        photos_lst = []
        for i, photo in enumerate(lst):
            photo_date = datetime.strftime(datetime.fromtimestamp(photo['date']), '%d-%m-%Y')
            photos_lst.append({
                'file_name': '_'.join([str(photo['likes']['count']), photo_date]),
                'size': photo['sizes'][-1]['type'],
                'url': photo['sizes'][-1]['url'],
            })
        return photos_lst


# Yandex API
class YaUploader:
    def __init__(self, token: str):
        self.ya_token = token

    def get_headers(self):
        return {'Content-type': 'application/json',
            'Authorization': f'OAuth {self.ya_token}'}

    def create_dir(self):
        """Метод создает папку на Яндекс-диске"""
        url = 'https://cloud-api.yandex.net/v1/disk/resources'
        headers = self.get_headers()
        params = {'path': 'Vk_photo'}
        response = requests.put(url, headers = headers, params = params)

    def upload(self, lst):
        """Метод загружает фотографии на яндекс диск"""
        pbar = tqdm(total = len(lst))
        url = 'https://cloud-api.yandex.net/v1/disk/resources/upload'
        headers = self.get_headers()
        for photo in lst:
            pbar.update(1)
            file_name = photo['file_name']
            path = f'Vk_photo/{file_name}.jpeg'
            params = {'path': path, 'url': photo['url']}
            response = requests.post(url, headers = headers, params = params)
        pbar.close()

In [31]:
if __name__ == '__main__':
    # токены из ini файла. при необходимости можно раскомментировать строки с инпутом
    config = ConfigParser()
    config.read('config.ini')
    vk_token = config['Tokens']['vk_token']
    ya_token = config['Tokens']['ya_token']
    # vk_token = input("Введите токен vk: ")
    # ya_token = input("Введите токен Яндекса: ")

    # создание объектов классов
    vk_result = VkGet(vk_token)
    uploader = YaUploader(ya_token)

    # Получение информации о пользователе и создание списка с фото
    while True:
        ask_1 = input('Вы хотите скачать фотографии из своего профиля? (да/нет) ').lower()
        if ask_1 == 'нет':
            user_id = input('Укажите id владельца альбома или его screen_name: ')
            if user_id.isalpha():
                owner_id = vk_result.get_user_id(user_id)
            else:
                owner_id = user_id
                break
        if ask_1 == 'да':
            owner_id = None
            break
        else:
            print('Вы ввели некорректное значение')
            continue
    
    while True:    
        ask_2 = int(input('Укажите количество фотографий, которое хотите скачать: '))
        if ask_2.isdigit():
            photo_lst = vk_result.lst_photos(vk_result.get_photos(owner_id=owner_id, count=ask_2))
            break
        else:
            print('Вы ввели некорректное значение')
            continue    


    #  создание папки, загрузка фотографий, формирование json
    uploader.create_dir()
    uploader.upload(photo_lst)
    json_photos('ben_photos', photo_lst)

100%|██████████| 5/5 [00:02<00:00,  1.96it/s]
